In [1]:
from pandas import read_feather, read_csv
from pandas import DataFrame
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [2]:
def distance(lat1, lng1, lat2, lng2):
    #return distance as meter if you want km distance, remove "* 1000"
    radius = 6371

    dLat = (lat2-lat1) * np.pi / 180
    dLng = (lng2-lng1) * np.pi / 180

    lat1 = lat1 * np.pi / 180
    lat2 = lat2 * np.pi / 180

    val = np.sin(dLat/2) * np.sin(dLat/2) + np.sin(dLng/2)\
    * np.sin(dLng/2) * np.cos(lat1) * np.cos(lat2)    
    ang = 2 * np.arctan2(np.sqrt(val), np.sqrt(1-val))
    return radius * ang

# Test Sample

In [3]:
# No missing values
test_df = read_csv('test.csv', parse_dates=["pickup_datetime"],
                  infer_datetime_format=True)
print(test_df.shape)
test_df.head()

(9914, 7)


,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44,-73.982524,40.751260,-73.979654,40.746139,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12,-73.981160,40.767807,-73.990448,40.751635,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12,-73.966046,40.789775,-73.988565,40.744427,1


In [4]:
lng1min = test_df.pickup_longitude.min()
lng2min = test_df.dropoff_longitude.min()
lat1min = test_df.pickup_latitude.min()
lat2min = test_df.dropoff_latitude.min()
#
lng1max = test_df.pickup_longitude.max()
lng2max = test_df.dropoff_longitude.max()
lat1max = test_df.pickup_latitude.max()
lat2max = test_df.dropoff_latitude.max()

# Train Sample 1

In [12]:
%%time
# Read a subsample (train1) of the original dataset with 55M rows
# It was obtained ramdomly using the command line subsample task:
# >subsample --reservoir -n 1000000 train.csv -r > train1.csv
train_df = read_feather('tmp/train1.feather')
train_df.head()

CPU times: user 66.5 ms, sys: 97.9 ms, total: 164 ms
Wall time: 680 ms


In [13]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 7 columns):
fare_amount          float32
pickup_datetime      datetime64[ns]
pickup_longitude     float32
pickup_latitude      float32
dropoff_longitude    float32
dropoff_latitude     float32
passenger_count      uint8
dtypes: datetime64[ns](1), float32(5), uint8(1)
memory usage: 55.3 MB


In [14]:
def transform(train_df):
    
    # Remove missing values
    train_df = train_df.dropna(how = 'any', axis = 'rows')

    # Remove absurd passenger_count
    train_df = train_df[(train_df['passenger_count'] >= 1) &
                    (train_df['passenger_count'] <= 6)]

    # Remove negative and extreme fare_amount values
    train_df = train_df[(train_df['fare_amount'] >= 2.5) & (train_df['fare_amount'] <= 300)]
    print('Maximum fare_amount: %.1f' % train_df['fare_amount'].max())

    # Remove no displacements
    train_df = train_df[(train_df['pickup_latitude'] != train_df['dropoff_latitude'])]
    train_df = train_df[(train_df['pickup_longitude'] != train_df['dropoff_longitude'])]

    # Remove absurd displacements
    train_df = train_df[(train_df['pickup_longitude'] >= lng1min) & (train_df['pickup_longitude'] <= lng1max)]
    train_df = train_df[(train_df['dropoff_longitude'] >= lng2min) & (train_df['dropoff_longitude'] <= lng2max)]
    train_df = train_df[(train_df['pickup_latitude'] >= lat1min) & (train_df['pickup_latitude'] <= lat1max)]
    train_df = train_df[(train_df['dropoff_latitude'] >= lat2min) & (train_df['dropoff_latitude'] <= lat2max)]

    # Create new features - distance
    train_df['dist'] = distance(train_df['pickup_latitude'], train_df['pickup_longitude'],
                                train_df['dropoff_latitude'], train_df['dropoff_longitude'])
    #train_df = train_df[train_df['dist'] < 100.01]

    # Create new features - dayofweek,hour,month,year
    train_df['dayofweek'] = train_df['pickup_datetime'].dt.dayofweek.astype('uint8')
    train_df['hour'] = train_df['pickup_datetime'].dt.hour.astype('uint8')
    train_df['month'] = train_df['pickup_datetime'].dt.month.astype('uint8')
    train_df['year'] = train_df['pickup_datetime'].dt.year.astype('uint16')

    # Create dataframes for the two periods
    P1 = train_df[(train_df['pickup_datetime'] < '2012-09-01')]
    P1 = P1.drop(['pickup_datetime'], axis=1)
    print(P1.shape)
    P2 = train_df.loc[(train_df['pickup_datetime'] >= '2012-09-01')]
    P2 = P2.drop(['pickup_datetime'], axis=1)
    print(P2.shape)

    # Save memory
    print(train_df.shape)
    print('Maximum ride distance: %.1f' % train_df['dist'].max())
    del train_df
    return P1,P2

In [15]:
P1, P2 = transform(train_df)

Maximum fare_amount: 300.0
(1099637, 11)
(827729, 11)
(1927366, 12)
Maximum ride distance: 115.2


In [16]:
P1.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,dist,dayofweek,hour,month,year
0,34.669998,-73.870819,40.773991,-73.999054,40.760658,1,10.900634,4,18,10,2009
2,3.300000,-73.980865,40.750500,-73.981079,40.755962,1,0.607685,5,1,2,2009
3,7.300000,-73.979965,40.743340,-73.988792,40.759567,1,1.951639,0,20,10,2011
8,4.100000,-74.002220,40.738979,-74.003189,40.732300,1,0.747204,4,11,8,2010
10,12.100000,-73.982155,40.772606,-73.968719,40.751492,1,2.606266,3,8,7,2012


# Model P1

In [17]:
from sklearn.model_selection import train_test_split

X = P1.iloc[:,1:].values
y = P1.iloc[:,0].values

seed = 101

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3, random_state=seed)

## GBM Scikit-Learn

In [25]:
%%time
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score

# To speed up GBM we can utilize warm_start building tree
# models incrementally with a handy for loop.
modelP1 = GradientBoostingRegressor(random_state=seed, max_depth = 7,
                                    learning_rate=0.05,
                                    subsample=0.5, warm_start=True, verbose=1)

for n_estimators in range(1, 80, 5):
    modelP1.set_params(n_estimators=n_estimators)
    modelP1.fit(X_train, y_train)

y_pred=modelP1.predict(X_test)
print('RMSE before gridsearch: %.3f' % np.sqrt(mean_squared_error(y_pred,y_test)).round(3))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1          62.5739           5.5182            0.00s
      Iter       Train Loss      OOB Improve   Remaining Time 
         2          57.5837           4.9741           24.04s
         3          52.9893           4.5235           17.87s
         4          49.7394           4.0513           12.09s
         5          45.2714           3.6986            6.07s
         6          42.3671           3.3209            0.00s
      Iter       Train Loss      OOB Improve   Remaining Time 
         7          39.0262           3.0021           24.45s
         8          36.1776           2.7095           18.18s
         9          34.4151           2.4394           12.10s
        10          31.6392           2.2232            6.07s
        11          29.8741           1.9963            0.00s
      Iter       Train Loss      OOB Improve   Remaining Time 
        12          27.8399           1.7989           24.81s
    

## XGBoost

In [26]:
%%time
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import make_scorer, mean_squared_error

modelP1 = xgb.XGBRegressor(gamma=0, objective="reg:linear",
                           nthread=-1)
modelP1.fit(X_train,y_train)
y_pred=modelP1.predict(X_test)
print('RMSE before gridsearch: %.3f' % np.sqrt(mean_squared_error(y_pred,y_test)).round(3))

RMSE before gridsearch: 3.402
CPU times: user 1min 28s, sys: 4.28 s, total: 1min 32s
Wall time: 22.9 s


### XGBoost with GridSearchCV

In [30]:
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

params = {'max_depth':[3,4],
          'n_estimators':[100],
          'min_child_weight':[1,2],
          'learning_rate':[0.05,.1],
          'colsample_bytree':[0.5,1],
          'gamma':[0,1]}

cvx = xgb.XGBRegressor(objective= "reg:linear",
                           nthread=-1)

modelP1 = GridSearchCV(cvx, param_grid=params,
                       scoring=make_scorer(mean_squared_error),
                       verbose=True)

modelP1.fit(X_train,y_train)
print(modelP1.best_params_)
y_pred = modelP1.predict(X_test)
print('RMSE after gridsearch: %.3f' % np.sqrt(mean_squared_error(y_pred,y_test)).round(3))

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed: 24.3min finished


{'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100}
RMSE after gridsearch: 3.503
CPU times: user 1h 13min 46s, sys: 6min 18s, total: 1h 20min 4s
Wall time: 24min 34s


In [29]:
?XGBRegressor

## GridSearch

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_dist = {"max_features": [4,5,6],
              "min_samples_split": [4,6,8],
              "min_samples_leaf": [1,2,3],
              "bootstrap": [True, False]}

rsearch = RandomizedSearchCV(modelP1, param_distributions=param_dist, n_jobs=-1, n_iter=20)
rsearch.fit(X_train,y_train)

In [ ]:
modelP1=rsearch.best_estimator_
print(modelP1)

In [ ]:
#modelP1 = ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=20,
#          max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
#          min_impurity_split=None, min_samples_leaf=3, min_samples_split=4,
#          min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
#          oob_score=False, random_state=101, verbose=0, warm_start=True)

modelP1 = ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=20,
          max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
          min_impurity_split=None, min_samples_leaf=1, min_samples_split=6,
          min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=-1,
          oob_score=False, random_state=101, verbose=0, warm_start=True)
modelP1.fit(X_train,y_train)

# Best model applyied on the test set
y_pred=modelP1.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test)).round(3)

# Model P2

In [ ]:
from sklearn.model_selection import train_test_split

X = P2.iloc[:,1:13].values
y = P2.iloc[:,0].values

seed = 101

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3, random_state=seed)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_score

modelP2 = ExtraTreesRegressor(random_state=seed, n_estimators=15,
                              max_depth = 15, n_jobs = -1, warm_start=True)

modelP2.fit(X_train,y_train)

scores = cross_val_score(modelP2, X_train, y_train, cv=3, scoring=make_scorer(mean_squared_error))
print('RMSE CV: %.3f +/- %.3f' % (np.sqrt(np.mean(scores)), np.sqrt(np.std(scores))))

y_pred=modelP2.predict(X_test)
print('RMSE: %.3f' % np.sqrt(mean_squared_error(y_pred,y_test)).round(3))

## GridSearch

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_dist = {"max_features": [4,5,6],
              "min_samples_split": [4,6,8],
              "min_samples_leaf": [1,2,3],
              "bootstrap": [True, False]}

rsearch = RandomizedSearchCV(modelP2, param_distributions=param_dist, n_jobs=-1, n_iter=20)
rsearch.fit(X_train,y_train)

In [ ]:
modelP2=rsearch.best_estimator_
print(modelP2)

In [ ]:
#modelP2 = ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=20,
#          max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
#          min_impurity_split=None, min_samples_leaf=1, min_samples_split=6,
#          min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
#          oob_score=False, random_state=101, verbose=0, warm_start=True)

modelP2 = ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
          min_impurity_split=None, min_samples_leaf=2, min_samples_split=6,
          min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=-1,
          oob_score=False, random_state=101, verbose=0, warm_start=True)

modelP2.fit(X_train,y_train)

# Best model applyied on the test set
y_pred=modelP2.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test)).round(3)

# Train Sample 2

In [ ]:
# Read the file
train_df = read_feather('tmp/train2.feather')

# Remove missing values
train_df = train_df.dropna(how = 'any', axis = 'rows')

# Remove absurd passenger_count
train_df = train_df[(train_df['passenger_count'] >= 1) &
                    (train_df['passenger_count'] <= 6)]

# Remove negative and extreme fare_amount values
train_df = train_df[(train_df['fare_amount'] >= 2.5) & (train_df['fare_amount'] <= 500)]

# Remove no displacements
train_df = train_df[(train_df['pickup_latitude'] != train_df['dropoff_latitude']) &
                    (train_df['pickup_longitude'] != train_df['dropoff_longitude'])]

# Remove absurd displacements
train_df = train_df[(train_df['pickup_longitude'] > lng2min) & (train_df['pickup_longitude'] < lng1max)]
train_df = train_df[(train_df['dropoff_longitude'] > lng2min) & (train_df['dropoff_longitude'] < lng1max)]
train_df = train_df[(train_df['pickup_latitude'] > lat2min) & (train_df['pickup_latitude'] < lat1max)]
train_df = train_df[(train_df['dropoff_latitude'] > lat2min) & (train_df['dropoff_latitude'] < lat1max)]

# Create new features - distance
train_df['dist'] = distance(train_df['pickup_latitude'], train_df['pickup_longitude'],
                            train_df['dropoff_latitude'], train_df['dropoff_longitude'])

# Create new features - dayofweek,hour,month,year
train_df['dayofweek'] = train_df['pickup_datetime'].dt.dayofweek.astype('uint8')
train_df['hour'] = train_df['pickup_datetime'].dt.hour.astype('uint8')
train_df['month'] = train_df['pickup_datetime'].dt.month.astype('uint8')
train_df['year'] = train_df['pickup_datetime'].dt.year.astype('uint16')

# Create dataframes for the two periods
P1 = train_df[(train_df['pickup_datetime'] < '2012-09-01')]
P1 = P1.drop(['pickup_datetime'], axis=1)
print(P1.shape)
P2 = train_df.loc[(train_df['pickup_datetime'] >= '2012-09-01')]
P2 = P2.drop(['pickup_datetime'], axis=1)
print(P2.shape)

# Save memory
print(train_df.shape)
del train_df

## P1

In [ ]:
X = P1.iloc[:,1:11].values
y = P1.iloc[:,0].values

for n_estimators in range(22, 32, 2):
    modelP1.set_params(n_estimators=n_estimators)
    modelP1.fit(X_train, y_train)
    
y_pred=modelP1.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test)).round(3)

## P2

In [ ]:
X = P2.iloc[:,1:11].values
y = P2.iloc[:,0].values

modelP2.set_params(n_estimators=20, random_state=101, n_jobs = -1, warm_start=True)
modelP2.fit(X, y)
y_pred=modelP2.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test)).round(3)

# Train Sample 3

In [ ]:
# Read the file
train_df = read_feather('tmp/train3.feather')

# Remove missing values
train_df = train_df.dropna(how = 'any', axis = 'rows')

# Remove absurd passenger_count
train_df = train_df[(train_df['passenger_count'] >= 1) &
                    (train_df['passenger_count'] <= 6)]

# Remove negative and extreme fare_amount values
train_df = train_df[(train_df['fare_amount'] >= 2.5) & (train_df['fare_amount'] <= 500)]

# Remove no displacements
train_df = train_df[(train_df['pickup_latitude'] != train_df['dropoff_latitude']) &
                    (train_df['pickup_longitude'] != train_df['dropoff_longitude'])]

# Remove absurd displacements
train_df = train_df[(train_df['pickup_longitude'] > lng2min) & (train_df['pickup_longitude'] < lng1max)]
train_df = train_df[(train_df['dropoff_longitude'] > lng2min) & (train_df['dropoff_longitude'] < lng1max)]
train_df = train_df[(train_df['pickup_latitude'] > lat2min) & (train_df['pickup_latitude'] < lat1max)]
train_df = train_df[(train_df['dropoff_latitude'] > lat2min) & (train_df['dropoff_latitude'] < lat1max)]

# Create new features - distance
train_df['dist'] = distance(train_df['pickup_latitude'], train_df['pickup_longitude'],
                            train_df['dropoff_latitude'], train_df['dropoff_longitude'])

# Create new features - dayofweek,hour,month,year
train_df['dayofweek'] = train_df['pickup_datetime'].dt.dayofweek.astype('uint8')
train_df['hour'] = train_df['pickup_datetime'].dt.hour.astype('uint8')
train_df['month'] = train_df['pickup_datetime'].dt.month.astype('uint8')
train_df['year'] = train_df['pickup_datetime'].dt.year.astype('uint16')

# Create dataframes for the two periods
P1 = train_df[(train_df['pickup_datetime'] < '2012-09-01')]
P1 = P1.drop(['pickup_datetime'], axis=1)
print(P1.shape)
P2 = train_df.loc[(train_df['pickup_datetime'] >= '2012-09-01')]
P2 = P2.drop(['pickup_datetime'], axis=1)
print(P2.shape)

# Save memory
print(train_df.shape)
del train_df

## P1

In [ ]:
X = P1.iloc[:,1:11].values
y = P1.iloc[:,0].values

for n_estimators in range(34, 44, 2):
    modelP1.set_params(n_estimators=n_estimators)
    modelP1.fit(X_train, y_train)
    
y_pred=modelP1.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test)).round(3)

## P2

In [ ]:
X = P2.iloc[:,1:11].values
y = P2.iloc[:,0].values

modelP2.set_params(n_estimators=25, random_state=101, n_jobs = -1, warm_start=True)
modelP2.fit(X, y)
y_pred=modelP2.predict(X_test)
np.sqrt(mean_squared_error(y_pred,y_test)).round(3)

## Make predictions on the test set

In [ ]:
# No missing values
test_df = read_csv('test.csv', parse_dates=["pickup_datetime"],
                  infer_datetime_format=True)

test_df['dist'] = distance(test_df['pickup_latitude'], test_df['pickup_longitude'],
                           test_df['dropoff_latitude'], test_df['dropoff_longitude'])

test_df['dayofweek'] = test_df['pickup_datetime'].dt.dayofweek #.astype('uint8')
test_df['hour'] = test_df['pickup_datetime'].dt.hour #.astype('uint8')
test_df['month'] = test_df['pickup_datetime'].dt.month #.astype('uint8')
test_df['year'] = test_df['pickup_datetime'].dt.year #.astype('uint8')
#test_df = test_df.drop(['passenger_count'], axis=1)
test_df.head()

In [ ]:
X_testF = test_df.iloc[:,[2,3,4,5,6,7,8,9,10,11]].values
y_predFP1 = modelP1.predict(X_testF).round(3)

In [ ]:
#X_testF = test_df.iloc[:,[2,3,4,5,6,7,8,9,10,11]].values
#y_predFP1 = modelP1.predict(X_testF).round(3)
y_predFP2 = modelP2.predict(X_testF).round(3)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter(y_predFP1, y_predFP2)
plt.xlabel('y_pred P1')
plt.ylabel('y_pred P2')
plt.xlim(0,150)
plt.ylim(0,150)
plt.show()

In [ ]:
#y_predFP2 = np.where(y_predFP2 > 150, 100, y_predFP2)
#y_predFP2 = np.where((y_predFP2 > 90) & (y_predFP1 < 40), 35, y_predFP2)
#y_predFP2 = np.where((y_predFP2 > 40) & (y_predFP1 < 24), 11, y_predFP2)

In [ ]:
#import matplotlib.pyplot as plt
#%matplotlib inline
#plt.scatter(y_predFP1, y_predFP2)
#plt.xlabel('y_pred P1')
#plt.ylabel('y_pred P2')
#plt.xlim(0,250)
#plt.ylim(0,250)
#plt.show()

In [ ]:
submission = DataFrame({'key': test_df.key, 'fare_amountP1': y_predFP1,
                        'fare_amountP2': y_predFP2},
                       columns = ['key', 'fare_amountP1', 'fare_amountP2'])

submission['fare_amount'] = np.where(test_df['pickup_datetime'] < '2012-09-01',
                                     submission['fare_amountP1'],
                                     submission['fare_amountP2'])

submission = submission.drop(['fare_amountP1','fare_amountP2'], axis=1)
submission.to_csv('submission.csv', index = False)
submission.head()

In [ ]:
testP1 = test_df[(test_df['pickup_datetime'] < '2012-09-01')]
print(testP1.shape)
testP2 = test_df[(test_df['pickup_datetime'] >= '2012-09-01')]
print(testP2.shape)